# **民族服饰分类**

In [1]:
# Importing PlaidML. Make sure you follow this order
import plaidml.keras

plaidml.keras.install_backend()
import os

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# then you can write your codes

from keras.preprocessing.image import ImageDataGenerator

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical

## Data crawler(*droped*)

## Import data

### Data enhancement

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [4]:
batch_size = 1

train_generator = train_datagen.flow_from_directory(
    'images/train',
    # target_size=(300, 300),
    batch_size=batch_size,
    # class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'images/test',
    # target_size=(150, 150),
    batch_size=batch_size,
    # class_mode='binary'
)

Found 1135 images belonging to 56 classes.
Found 168 images belonging to 56 classes.


In [5]:
from keras.layers import Conv2D, BatchNormalization, PReLU, add, ReLU


def Conv_BN_Relu(filters, kernel_size, strides, input_layer):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_initializer="he_normal")(input_layer)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x


def residual(input_x, filters, down_sample=False):

    strides = [2, 1] if down_sample else [1, 1]
    kernel_size = (3,3)
    INIT_SCHEME = "he_normal"

    # 主路
    x = Conv_BN_Relu(filters, kernel_size, strides[0], input_x)
    x = Conv_BN_Relu(filters, kernel_size, strides[1], x)

    if down_sample:

        # 支路下采样
        input_x = Conv_BN_Relu(filters, (1, 1), 2, input_x)

    # 输出
    y = add([x, input_x])

    return y


In [6]:
from keras.layers import Input, MaxPooling2D, GlobalAveragePooling2D, Flatten, AveragePooling2D, Reshape
from keras.layers import Dense, Dropout, Softmax
from keras.models import Model

# input_layer = Input((256, 256, 3))
input_layer = Input((28,28,1))
conv1 = Conv_BN_Relu(64, (7, 7), 1, input_layer)
conv1_Maxpooling = MaxPooling2D((3, 3), strides=2, padding='same')(conv1)

# conv2_x
x = residual(conv1_Maxpooling, 64, True)
x = residual(x, 64, 'a')

# conv3_x
x = residual(x, 128, 'b')
x = residual(x, 128, 'a')

# conv4_x
x = residual(x, 256, 'b')
x = residual(x, 256, 'a')

# conv5_x
x = residual(x, 512, 'b')
x = residual(x, 512, 'a')

# 最后一层
x = GlobalAveragePooling2D()(x)
x = Reshape((512,1))(x)
x = Flatten()(x)
# x = PReLU()(x)
x = Dense(10)(x)
y = Softmax(axis=-1)(x)

model = Model([input_layer], [y])

model.summary()

INFO:plaidml:Opening device "llvm_cpu.0"


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 28, 28, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [7]:
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0


X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# 调整标签为onehot
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [8]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(X_train, Y_train, epochs=16, batch_size=128)
# 评估模型
evaluate_res = model.evaluate(x=X_test, y=Y_test, batch_size=128)

# 输出评估结果
print(evaluate_res)

Epoch 1/16
  512/60000 [..............................] - ETA: 2:34:02 - loss: nan - acc: 0.0645


KeyboardInterrupt



In [ ]:
classes = 56

model = resnet_model(classes, train_generator.image_shape)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

In [ ]:
model.fit_generator(
      train_generator,
      # steps_per_epoch=2000,
      epochs=50,
      validation_data=validation_generator,
      # validation_steps=800
)